In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
import scipy as sp
from scipy import spatial
import os
import merge_bubbles_3d_v2 as merge
from matplotlib.colors import hsv_to_rgb
import advect_bubbles_3D_eval as adv
import pickle

# N_realisation = 47
# np.random.seed(N_realisation)  

Generate uniformly distributed particles of a thin volume, resembling a line plume, then put particles into grid cells. Two stagger sets of grids, `gridA` and `gridB`. 

Params to set:
- number of particles
- St0, Fr
- critical collision distance

In [ ]:
# FOR actual simulation
# N0_particle = V_gas / (4/3 * np.pi * a_0 **3)  # Number of particles in the domain
N0_particle = 150

# line plume thickness
plume_w = 0.1

XP0 = np.random.uniform((2-plume_w), 2, N0_particle) 
YP0 = np.random.uniform(-2, 2, N0_particle)
ZP0 = np.random.uniform(-6, 2, N0_particle)

St0 = 0.2
R_collision = 0.02
Fr = 0.5

xl, xr = -6, 2
yl, yr = -2, 2
zd, zu = -6, 2

dx_col, dy_col, dz_col = 1, 1, 1
Nx_col, Ny_col, Nz_col = 8, 4, 8
gridA_xc, gridA_yc, gridA_zc = np.meshgrid(np.linspace(xl+dx_col/2, xr-dx_col/2, Nx_col), 
                                           np.linspace(yl+dy_col/2, yr-dy_col, Ny_col), 
                                           np.linspace(zd+dz_col/2, zu-dz_col/2, Nz_col), indexing='ij')
gridB_xc, gridB_yc, gridB_zc = np.meshgrid(np.linspace(xl, xr, Nx_col+1), np.linspace(yl, yr, Ny_col+1), np.linspace(zd, zu, Nz_col+1), indexing='ij')

# Plot grid lines
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

ax.scatter(XP0, YP0, ZP0,s=1, c='k', marker='o')
ax.set_xlim(-6, 2)
ax.set_ylim(-2, 2)
ax.set_zlim(-6, 2)


plt.show()

Initialise the Lagrangian table

In [ ]:
Bubbles_df_ini = np.zeros((N0_particle, 16), dtype=np.float64)

# first column is bubbles ID
Bubbles_df_ini[:, 0] = np.arange(N0_particle)

# second column is bubbles X position
Bubbles_df_ini[:, 1] = XP0.flatten().astype(float)

# third column is bubbles Y position   
Bubbles_df_ini[:, 2] = YP0.flatten().astype(float)

# fourth column is bubbles Z position
Bubbles_df_ini[:, 3] = ZP0.flatten().astype(float)

# fourth column is bubbles x-velocity
Bubbles_df_ini[:, 4] = np.zeros(N0_particle)

# fifth column is bubbles y-velocity
Bubbles_df_ini[:, 5] = np.zeros(N0_particle)

# sixth column is bubbles z-velocity
Bubbles_df_ini[:, 6] = np.zeros(N0_particle)

# sixth column is bubbles Stokes number
Bubbles_df_ini[:, 7] = St0

# seventh column is bubbles slaved status
Bubbles_df_ini[:, 8] = False

# eighth column is bubbles gridA x-index
Bubbles_df_ini[:, 9] = np.floor((Bubbles_df_ini[:, 1] - xl ) / dx_col).astype(int)

# ninth column is bubbles gridA y-index
Bubbles_df_ini[:, 10] = np.floor((Bubbles_df_ini[:, 2] - yl ) / dy_col).astype(int)

# tenth column is bubbles gridA z-index
Bubbles_df_ini[:, 11] = np.floor((Bubbles_df_ini[:, 3] - zd ) / dz_col).astype(int)

# tenth column is bubbles gridB x-index
Bubbles_df_ini[:, 12] = np.floor((Bubbles_df_ini[:, 1]- xl + dx_col / 2) / dx_col).astype(int)

# eleventh column is bubbles gridB y-index	
Bubbles_df_ini[:, 13] = np.floor((Bubbles_df_ini[:, 2]- yl + dy_col / 2) / dy_col).astype(int)

# twelfth column is bubbles gridB z-index
Bubbles_df_ini[:, 14] = np.floor((Bubbles_df_ini[:, 3]- zd + dz_col / 2) / dz_col).astype(int)

# the last column tracks if fraction of bubbles have visited the core
Bubbles_df_ini[:, 15] = np.zeros(N0_particle)

# ditch the ones about to leave the domain 
Bubbles_df_ini = Bubbles_df_ini[(Bubbles_df_ini[:, 1] >= xl) & (Bubbles_df_ini[:, 1] < xr) 
                                & (Bubbles_df_ini[:, 2] >= yl) & (Bubbles_df_ini[:, 2] < yr)
                                & (Bubbles_df_ini[:, 3] >= zd) & (Bubbles_df_ini[:, 3] < zu)]

Bubbles coming from upstream reservoir 

Pool - Bubble coming from below


In [ ]:
from matplotlib.path import Path
path = 'velocity_fields/alpha_0.4_2D/'


geometry_2D = np.load(path + 'geometry.npy')

_, _, _, x_ring_2d, y_ring_2d = geometry_2D.T

def mark_inRing(before_adv_df, after_adv_df):
    '''
    after_adv_df : (n_bubble, 3), (x, y, z)-positions of bubbles after advection
    '''
    x_2d = after_adv_df[:, 1]
    y_2d = np.sqrt(after_adv_df[:, 2]**2 + after_adv_df[:, 3]**2)
    inside = Path(np.vstack([x_ring_2d, y_ring_2d]).T).contains_points(np.vstack([x_2d, y_2d]).T)
    before_adv_df[inside, 15] = 1

    return None



In [ ]:

def record_deposit_bubbles(current_time, before_df, after_df, deposit_dict, ceiling_height: float, incore_thresh: float =0):
    '''
    before_df : (n_bubble, 16) np.array, the last column tracks volume fraction of the bubble that has been in the core
    after_df : (n_bubble, 3), (x, y, z)-positions of bubbles after advection
    modify deposit_dict in-place
    '''

    # filter out the bubbles that have been in the core
    # filter out bubbles that deposit 
    filtered_before_df = before_df[(before_df[:, 15] >= incore_thresh) & (after_df[:, 2] >= ceiling_height), :]
    

    if filtered_before_df.shape[0] == 0:
        deposit_dict[current_time] = None
        return None

    else:
        after_df = after_df[(before_df[:, 15] >= incore_thresh) & (after_df[:, 2] > ceiling_height), :]

        # linear interpolate to find the x, y-location of the deposited bubbles
        before_df = filtered_before_df
        dep_x_location = (before_df[:, 1] * (after_df[:, 2] - ceiling_height) + after_df[:, 0] * (ceiling_height - before_df[:, 3])) / (after_df[:, 2] - before_df[:, 3])
        dep_y_location = (before_df[:, 2] * (after_df[:, 2] - ceiling_height) + after_df[:, 1] * (ceiling_height - before_df[:, 3])) / (after_df[:, 2] - before_df[:, 3])
        dep_z_location = after_df[:, 2]

        deposit_dict[current_time] = np.hstack((dep_x_location, dep_y_location, dep_z_location,  before_df[:, 7], before_df[:, 15]))

        return None

# test_before_df = np.hstack((Bubbles_df_to_adv, np.random.uniform(0, 1, (Bubbles_df_to_adv.shape[0], 1))))



In [ ]:
tNow = 0
dt = 0.03
tEnd = 8
st_lim = 7
R_bounce = R_collision

N_tot = int((tEnd - tNow) / dt + 1)
N_mergeCycle = np.zeros(2, dtype=int)
# N_feedCycle = N_tot * np.ones(2, dtype=int)
# Nbub_already_supplemented = np.zeros(2)

# W_rising = St0_pool / (Fr**2)  # normalized by U_f
# flux_from_pool = N0_particle_pool * W_rising / (zd - z_far)  # normalized by 1/t_f
# N_in_from_pool = flux_from_pool * dt  # number of bubbles injected from the pool per time step

Bubbles_df_after_merge = Bubbles_df_ini.copy()
# Res_df_ini = Res_df_ini_0.copy()
# Pool_df_ini = Pool_df_ini_0.copy()
# Pool_df_ini[:, 6] = W_rising

V, H = np.mgrid[0:1:301j, 0:1:301j]
colors_hsv = np.dstack((H, np.ones_like(H), V))
colors_rgb = hsv_to_rgb(colors_hsv)

fig, ax = plt.subplots()
ax = fig.add_subplot(111, projection='3d')

collision_series = []
masters_slaves_series = []
# influx_series = []
deposit_dict = {}
folder_path = 'lineplume_sims_3D/temp'

while tNow < tEnd:

    
    Bubbles_df_to_adv = Bubbles_df_after_merge.copy()
    # # updated_states = np.asarray(pab3d.main(initial_states=Bubbles_df_to_adv[:, 1:8], t0=tNow, tf=tNow+dt), dtype=np.float64).T
    updated_states = adv.advect_bubbles(Bubbles_df_to_adv, t0=tNow, tf=tNow+dt,plot_path=False, this_ax=None)
    # update inRing status
    mark_inRing(Bubbles_df_to_adv, updated_states)
    # update deposited bubbles
    record_deposit_bubbles(tNow, Bubbles_df_to_adv, updated_states[:, :3], deposit_dict, ceiling_height=2)

    Bubbles_df_before_merge = Bubbles_df_to_adv.copy()

    Bubbles_df_after_merge, collision_list, masters_slaves_list = merge.merge_package(Bubbles_df_before_merge=Bubbles_df_to_adv, advected_states=updated_states, 
                                            gridA_size=gridA_xc.shape, gridB_size=gridB_xc.shape, boundaries=(xl, xr, yl, yr, zd, zu), cell_size=(dx_col, dy_col, dz_col),
                                            R_collision=R_collision, st_lim=st_lim, R_bounce=R_bounce, stick_bool=False, bounce_bool=True,
                                            merge_method='volume-weighted', timeNow=tNow, this_ax=ax, color=colors_rgb[N_mergeCycle[0], N_mergeCycle[0], :])
    
    collision_series.append(collision_list)
    if len(masters_slaves_list) > 0:
        masters_slaves_series.append(masters_slaves_list)
    else:
        masters_slaves_series.append(None)
    tNow += dt
    N_mergeCycle += 1

    plt.savefig(folder_path + '/frame_{}.png'.format(N_mergeCycle[0]), dpi=300)
    np.save(os.path.join(folder_path, 'frame_{}.npy'.format(N_mergeCycle[0])), Bubbles_df_after_merge[:, 0:8])
    # plt.show()
    plt.close()



file_path = os.path.join(folder_path, 'collision_series.pkl')

with open(file_path, 'wb') as f:
    pickle.dump(collision_series, f)

with open(os.path.join(folder_path, 'masters_slaves_series.pkl'), 'wb') as f:
    pickle.dump(masters_slaves_series, f)

# with open(os.path.join(folder_path, 'influx_series.pkl'), 'wb') as f:
#     pickle.dump(influx_series, f)

with open(os.path.join(folder_path, 'deposit_dict.pkl'), 'wb') as f:
    pickle.dump(deposit_dict, f)


In [ ]:
# import parallel_advect_res_bubbles_3d as parb3d
# Res_df_ini = Res_df_ini_0.copy()
# Res_df_ini = np.hstack((Res_df_ini, St0_res * np.ones((Res_df_ini.shape[0], 1), dtype=np.float64)))
# input = Res_df_ini[:, 1:]
# out = np.asarray(parb3d.main(initial_states=input, t0=0, tf=0.3)).T

In [ ]:
file_path = os.path.join(folder_path, 'collision_series.pkl')

with open(file_path, 'wb') as f:
    pickle.dump(collision_series, f)

with open(os.path.join(folder_path, 'masters_slaves_series.pkl'), 'wb') as f:
    pickle.dump(masters_slaves_series, f)

# with open(os.path.join(folder_path, 'influx_series.pkl'), 'wb') as f:
#     pickle.dump(influx_series, f)

with open(os.path.join(folder_path, 'deposit_dict.pkl'), 'wb') as f:
    pickle.dump(deposit_dict, f)

collision heatmap

In [ ]:
# collision_box = {(x, y, z): 0 for x in range(101) for y in range(101) for z in range(101)}

# for collision in collision_series:
#     x_s = np.asarray([c[0] for c in collision])
#     y_s = np.asarray([c[1] for c in collision])
#     z_s = np.asarray([c[2] for c in collision])
#     j_s = np.floor((x_s - xl) / 0.04).astype(int)
#     i_s = np.floor((y_s - yl) / 0.04).astype(int)
#     k_s = np.floor((z_s - zd) / 0.04).astype(int)

#     coordinates = np.column_stack((i_s, j_s, k_s))

#     unique_coords, counts = np.unique(coordinates, axis=0, return_counts=True)

#     # create a dictionary 
#     coll_box_counts = dict(zip(map(tuple, unique_coords), counts))

#     for (i, j, k), count in coll_box_counts.items():
#         collision_box[(i, j, k)] += count

# counts_array = np.array(list(collision_box.values())).reshape(101, 101, 101).astype(float)
# counts_array /= np.max(counts_array)


# import pyvista as pv
# volume = pv.wrap(counts_array)
# # Plot the volume
# plotter = pv.Plotter()
# plotter.add_volume(volume, cmap="viridis")
# plotter.show()